# Store alaska inventory as parquet format

In [3]:
# seisbench planetary computer environment
!mamba env create --quiet -f ../seisbench-testing/environment.yml

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

done
Installing pip dependencies: ...working... done


## QuakeML

Reading quakeML is slow. We can extract relevant data from this file into a tablular format and save to a more cloud-friendly performant format like Parquet https://parquet.apache.org/documentation/latest/ 

In [40]:
import obspy
import obsplus
import pandas as pd
from obspy.clients.fdsn import Client
from datetime import timedelta

In [4]:
%%time

cat = obspy.core.event.read_events('../alaska_catalog/data_acquisition/catalog_css/2018_10/XO_2018_10.quakeml') #obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2020-11-19T01:12:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
print(cat)

/srv/conda/envs/seisbench/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2020-11-19T01:12:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


437 Event(s) in Catalog:
2018-10-01T06:02:23.795000Z | +57.940, -153.929 | 2.1 ml | manual
2018-10-01T08:35:07.702000Z | +57.139, -156.563 | 2.5 ml | manual
...
2018-10-31T11:05:22.076000Z | +56.208, -148.992 | 2.4 ml | manual
2018-10-31T20:03:47.517000Z | +55.858, -149.523 | 2.4 ml | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'
CPU times: user 48.9 s, sys: 447 ms, total: 49.4 s
Wall time: 49.4 s


In [36]:
type(cat)

obspy.core.event.catalog.Catalog

## NOTE: use obsplus package to facilitate dataframe conversions

## Events

In [7]:
%%time

dfE = obsplus.events_to_df(cat)
dfE.head()

CPU times: user 5.08 s, sys: 88 ms, total: 5.17 s
Wall time: 5.16 s


,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,stations
0,2018-10-01 06:02:23.795,57.9402,-153.9286,54300.0,2.1,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cl8u67g,NaN,...,0.83,97.0,76.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, AU22, AUE, AUJK, AUSB, AUW, AUWS, B..."
1,2018-10-01 08:35:07.702,57.1389,-156.5634,81400.0,2.5,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cla82sf,NaN,...,1.60,87.0,72.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ANCK, ANPK, BRSE, CHGN, CNP, EP15, EP22, EP23,..."
2,2018-10-01 09:42:17.906,58.2321,-155.1851,3400.0,1.4,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018clav17c,NaN,...,0.42,26.0,18.0,200.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, EP15, ET17, ET19, ET20, KABU, KAHG,..."
3,2018-10-01 09:55:56.347,55.8944,-149.8541,12000.0,2.8,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018claxwm6,NaN,...,0.89,55.0,45.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BAL, BARK, BERG, BRSE, CNP, CRQ, EYAK, GLI, HI..."
4,2018-10-01 14:10:56.076,55.6810,-149.2303,12200.0,2.7,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cldnfg8,NaN,...,0.88,61.0,43.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BRSE, CNP, FID, GHO, HIN, ILSW, KAKN, KD01, KD..."


In [11]:
dfE.iloc[0]

time                                             2018-10-01 06:02:23.795000
latitude                                                            57.9402
longitude                                                         -153.9286
depth                                                               54300.0
magnitude                                                               2.1
event_description                                                          
associated_phase_count                                                  0.0
azimuthal_gap                                                           NaN
event_id                     quakeml:earthquake.alaska.edu/event/018cl8u67g
horizontal_uncertainty                                                  NaN
local_magnitude                                                         2.1
moment_magnitude                                                        NaN
duration_magnitude                                                      NaN
magnitude_ty

In [8]:
# ArrowInvalid: Casting from timestamp[ns] to timestamp[us] would lose data: 1610761489008999936
# https://github.com/pandas-dev/pandas/issues/31572 
# df.to_parquet('XO_2018_10.parquet')
dfE.to_parquet('XO_2018_10_events.parquet', version='2.6')

In [9]:
# Round trip

df = pd.read_parquet('XO_2018_10_events.parquet')
df.head()

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,stations
0,2018-10-01 06:02:23.795,57.9402,-153.9286,54300.0,2.1,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cl8u67g,NaN,...,0.83,97.0,76.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, AU22, AUE, AUJK, AUSB, AUW, AUWS, B..."
1,2018-10-01 08:35:07.702,57.1389,-156.5634,81400.0,2.5,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cla82sf,NaN,...,1.60,87.0,72.0,300.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ANCK, ANPK, BRSE, CHGN, CNP, EP15, EP22, EP23,..."
2,2018-10-01 09:42:17.906,58.2321,-155.1851,3400.0,1.4,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018clav17c,NaN,...,0.42,26.0,18.0,200.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"ACH, ANCK, EP15, ET17, ET19, ET20, KABU, KAHG,..."
3,2018-10-01 09:55:56.347,55.8944,-149.8541,12000.0,2.8,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018claxwm6,NaN,...,0.89,55.0,45.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BAL, BARK, BERG, BRSE, CNP, CRQ, EYAK, GLI, HI..."
4,2018-10-01 14:10:56.076,55.6810,-149.2303,12200.0,2.7,,0.0,NaN,quakeml:earthquake.alaska.edu/event/018cldnfg8,NaN,...,0.88,61.0,43.0,25000.0,2021-01-16 01:44:49.008999936,"Alaska Earthquake Center, UAF/GI",AK,2021-01-16 01:44:49.009,1,"BRSE, CNP, FID, GHO, HIN, ILSW, KAKN, KD01, KD..."


## Arrivals

In [15]:
%%time 

dfA = obsplus.arrivals_to_df(cat)
dfA

CPU times: user 8.63 s, sys: 7.97 ms, total: 8.64 s
Wall time: 8.64 s


,resource_id,seed_id,pick_id,phase,time_correction,azimuth,distance,takeoff_angle,time_residual,horizontal_slowness_residual,...,earth_model_id,creation_time,author,agency_id,network,station,location,channel,origin_id,origin_time
0,quakeml:earthquake.alaska.edu/arrival/137,XO.KD02.--.HHE,quakeml:earthquake.alaska.edu/pick/137,S,NaN,96.08,0.680,NaN,-0.072,NaN,...,,2020-11-12 20:08:35.813,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHE,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
1,quakeml:earthquake.alaska.edu/arrival/98,XO.KD05.--.HHZ,quakeml:earthquake.alaska.edu/pick/98,P,NaN,103.73,0.284,NaN,-0.533,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KD05,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
2,quakeml:earthquake.alaska.edu/arrival/99,XO.KT06.--.HHZ,quakeml:earthquake.alaska.edu/pick/99,P,NaN,187.97,0.291,NaN,-0.252,NaN,...,,2020-11-13 01:05:19.635,"Alaska Earthquake Center, UAF/GI",AK,XO,KT06,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
3,quakeml:earthquake.alaska.edu/arrival/100,XO.KT07.--.HHZ,quakeml:earthquake.alaska.edu/pick/100,P,NaN,184.02,0.404,NaN,-0.166,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KT07,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
4,quakeml:earthquake.alaska.edu/arrival/101,AK.R18K.--.BHZ,quakeml:earthquake.alaska.edu/pick/101,P,NaN,217.02,0.467,NaN,-0.377,NaN,...,,2018-10-05 17:57:13.748,"Alaska Earthquake Center, UAF/GI",AK,AK,R18K,--,BHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28346,quakeml:earthquake.alaska.edu/arrival/27951,XO.KD01.--.HHE,quakeml:earthquake.alaska.edu/pick/27951,S,NaN,319.33,2.394,NaN,-2.320,NaN,...,,2020-12-19 02:31:29.641,"Alaska Earthquake Center, UAF/GI",AK,XO,KD01,--,HHE,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28347,quakeml:earthquake.alaska.edu/arrival/27952,XO.WD55.--.HH1,quakeml:earthquake.alaska.edu/pick/27952,S,NaN,269.35,2.328,NaN,1.813,NaN,...,,2020-12-19 02:33:08.591,"Alaska Earthquake Center, UAF/GI",AK,XO,WD55,--,HH1,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28348,quakeml:earthquake.alaska.edu/arrival/27953,II.KDAK.10.BH1,quakeml:earthquake.alaska.edu/pick/27953,S,NaN,320.25,2.552,NaN,-2.620,NaN,...,,2020-12-19 02:31:37.285,"Alaska Earthquake Center, UAF/GI",AK,II,KDAK,10,BH1,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517
28349,quakeml:earthquake.alaska.edu/arrival/27954,XO.KD02.--.HHN,quakeml:earthquake.alaska.edu/pick/27954,S,NaN,320.77,2.637,NaN,-1.081,NaN,...,,2020-12-19 02:33:30.545,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHN,quakeml:earthquake.alaska.edu/origin/018dz3ax1b,2018-10-31 20:03:47.517


In [12]:
dfA.iloc[0]

resource_id                           quakeml:earthquake.alaska.edu/arrival/137
seed_id                                                          XO.KD02.--.HHE
pick_id                                  quakeml:earthquake.alaska.edu/pick/137
phase                                                                         S
time_correction                                                             NaN
azimuth                                                                   96.08
distance                                                                   0.68
takeoff_angle                                                               NaN
time_residual                                                            -0.072
horizontal_slowness_residual                                                NaN
backazimuth_residual                                                        NaN
time_weight                                                               0.125
horizontal_slowness_weight              

In [17]:
dfA.to_parquet('XO_2018_10_arrivals.parquet', version='2.6')

In [19]:
# round trip
df = pd.read_parquet('XO_2018_10_arrivals.parquet')
df.head()

,resource_id,seed_id,pick_id,phase,time_correction,azimuth,distance,takeoff_angle,time_residual,horizontal_slowness_residual,...,earth_model_id,creation_time,author,agency_id,network,station,location,channel,origin_id,origin_time
0,quakeml:earthquake.alaska.edu/arrival/137,XO.KD02.--.HHE,quakeml:earthquake.alaska.edu/pick/137,S,NaN,96.08,0.680,NaN,-0.072,NaN,...,,2020-11-12 20:08:35.813,"Alaska Earthquake Center, UAF/GI",AK,XO,KD02,--,HHE,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
1,quakeml:earthquake.alaska.edu/arrival/98,XO.KD05.--.HHZ,quakeml:earthquake.alaska.edu/pick/98,P,NaN,103.73,0.284,NaN,-0.533,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KD05,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
2,quakeml:earthquake.alaska.edu/arrival/99,XO.KT06.--.HHZ,quakeml:earthquake.alaska.edu/pick/99,P,NaN,187.97,0.291,NaN,-0.252,NaN,...,,2020-11-13 01:05:19.635,"Alaska Earthquake Center, UAF/GI",AK,XO,KT06,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
3,quakeml:earthquake.alaska.edu/arrival/100,XO.KT07.--.HHZ,quakeml:earthquake.alaska.edu/pick/100,P,NaN,184.02,0.404,NaN,-0.166,NaN,...,,2020-10-02 22:32:18.450,"Alaska Earthquake Center, UAF/GI",AK,XO,KT07,--,HHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795
4,quakeml:earthquake.alaska.edu/arrival/101,AK.R18K.--.BHZ,quakeml:earthquake.alaska.edu/pick/101,P,NaN,217.02,0.467,NaN,-0.377,NaN,...,,2018-10-05 17:57:13.748,"Alaska Earthquake Center, UAF/GI",AK,AK,R18K,--,BHZ,quakeml:earthquake.alaska.edu/origin/018cl8u67g,2018-10-01 06:02:23.795


## Picks

In [14]:
%%time
dfP = obsplus.picks_to_df(cat)
dfP

CPU times: user 2.98 s, sys: 1 µs, total: 2.98 s
Wall time: 2.98 s


,resource_id,time,seed_id,filter_id,method_id,horizontal_slowness,backazimuth,onset,phase_hint,polarity,...,agency_id,event_id,network,station,location,channel,uncertainty,lower_uncertainty,upper_uncertainty,confidence_level
0,quakeml:earthquake.alaska.edu/pick/137,2018-10-01 06:02:47.026,XO.KD02.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD02,--,HHE,NaN,NaN,NaN,NaN
1,quakeml:earthquake.alaska.edu/pick/98,2018-10-01 06:02:32.200,XO.KD05.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD05,--,HHZ,NaN,NaN,NaN,NaN
2,quakeml:earthquake.alaska.edu/pick/99,2018-10-01 06:02:32.539,XO.KT06.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT06,--,HHZ,NaN,NaN,NaN,NaN
3,quakeml:earthquake.alaska.edu/pick/100,2018-10-01 06:02:33.620,XO.KT07.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT07,--,HHZ,NaN,NaN,NaN,NaN
4,quakeml:earthquake.alaska.edu/pick/101,2018-10-01 06:02:34.048,AK.R18K.--.BHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,AK,R18K,--,BHZ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28346,quakeml:earthquake.alaska.edu/pick/27951,2018-10-31 20:04:46.407,XO.KD01.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,KD01,--,HHE,NaN,NaN,NaN,NaN
28347,quakeml:earthquake.alaska.edu/pick/27952,2018-10-31 20:04:48.557,XO.WD55.--.HH1,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,WD55,--,HH1,NaN,NaN,NaN,NaN
28348,quakeml:earthquake.alaska.edu/pick/27953,2018-10-31 20:04:49.984,II.KDAK.10.BH1,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,II,KDAK,10,BH1,NaN,NaN,NaN,NaN
28349,quakeml:earthquake.alaska.edu/pick/27954,2018-10-31 20:04:53.576,XO.KD02.--.HHN,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018dz3ax1b,XO,KD02,--,HHN,NaN,NaN,NaN,NaN


In [16]:
dfP.iloc[0]

resource_id                    quakeml:earthquake.alaska.edu/pick/137
time                                       2018-10-01 06:02:47.026000
seed_id                                                XO.KD02.--.HHE
filter_id                                                            
method_id                                                            
horizontal_slowness                                               NaN
backazimuth                                                       NaN
onset                                                                
phase_hint                                                           
polarity                                                             
evaluation_mode                                                manual
event_time                                 2018-10-01 06:02:23.795000
evaluation_status                                                    
creation_time                              2020-11-12 20:08:35.813000
author              

In [20]:
dfP.to_parquet('XO_2018_10_picks.parquet', version='2.6')
df = pd.read_parquet('XO_2018_10_picks.parquet')
df.head()

,resource_id,time,seed_id,filter_id,method_id,horizontal_slowness,backazimuth,onset,phase_hint,polarity,...,agency_id,event_id,network,station,location,channel,uncertainty,lower_uncertainty,upper_uncertainty,confidence_level
0,quakeml:earthquake.alaska.edu/pick/137,2018-10-01 06:02:47.026,XO.KD02.--.HHE,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD02,--,HHE,NaN,NaN,NaN,NaN
1,quakeml:earthquake.alaska.edu/pick/98,2018-10-01 06:02:32.200,XO.KD05.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KD05,--,HHZ,NaN,NaN,NaN,NaN
2,quakeml:earthquake.alaska.edu/pick/99,2018-10-01 06:02:32.539,XO.KT06.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT06,--,HHZ,NaN,NaN,NaN,NaN
3,quakeml:earthquake.alaska.edu/pick/100,2018-10-01 06:02:33.620,XO.KT07.--.HHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,XO,KT07,--,HHZ,NaN,NaN,NaN,NaN
4,quakeml:earthquake.alaska.edu/pick/101,2018-10-01 06:02:34.048,AK.R18K.--.BHZ,,,NaN,NaN,,,,...,AK,quakeml:earthquake.alaska.edu/event/018cl8u67g,AK,R18K,--,BHZ,NaN,NaN,NaN,NaN


In [21]:
!ls -ltrh XO*

-rw-rw-r-- 1 jovyan users   30M Jan 19 00:46 XO_2018_10.quakeml
-rw-rw-r-- 1 jovyan users   85K Jan 19 16:46 XO_2018_10.parquet
-rw-r--r-- 1 jovyan users   85K Jan 19 18:34 XO_2018_10_events.parquet
-rw-r--r-- 1 jovyan users 1016K Jan 19 18:40 XO_2018_10_arrivals.parquet
-rw-r--r-- 1 jovyan users  772K Jan 19 18:41 XO_2018_10_picks.parquet


## Stations and response

In [44]:
# obspy.core.event.catalog.Catalog doesn't include lat,lon, etc.
# better to pass obspy.core.inventory.inventory.Inventory from IRIS search...
dfS = obsplus.stations_to_df(cat)
dfS

,network,station,location,channel,seed_id,latitude,longitude,elevation,depth,azimuth,dip,sample_rate,start_date,end_date
0,AK,BAGL,--,BHZ,AK.BAGL.--.BHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,AK,BAL,--,BHZ,AK.BAL.--.BHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,AK,BARK,--,BHN,AK.BARK.--.BHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,AK,BARK,--,BHZ,AK.BARK.--.BHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,AK,BCP,--,BHZ,AK.BCP.--.BHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,XO,WS75,--,HH2,XO.WS75.--.HH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
788,XO,WS75,--,HHZ,XO.WS75.--.HHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
789,XV,FNN1,--,HHZ,XV.FNN1.--.HHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
790,XV,FNN2,--,HHZ,XV.FNN2.--.HHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [33]:
# NOTE: not sorted in chronological order... how sorted?
print(dfE.time.min())
print(dfE.iloc[0].time)

2018-10-01 03:06:24.332000
2018-10-01 06:02:23.795000


In [55]:
network = "XO"
channel = "BH*,BN*,EH*,HH*,HN*,SH*"
#station = None
station = ','.join(dfS.query(' network == "XO" ').station.unique().tolist())
starttime = dfE.time.min()


inventory = Client("iris").get_stations(starttime = starttime, 
                                           endtime  = starttime + timedelta(weeks=4),
                                           network = network,
                                           channel = channel,
                                           station = station,
                                           level = 'response',
                                           location = '')
    
print(type(inventory))
print(inventory)

<class 'obspy.core.inventory.inventory.Inventory'>
Inventory created at 2022-01-19T19:26:26.122000Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.48
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2018-10-01...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			XO
		Stations (89):
			XO.EP14 (Igiugig, AK)
			XO.EP15 (Kulic Lake, AK)
			XO.EP16 (Levelock, AK)
			XO.EP21 (Egegik, AK)
			XO.EP22 (Shoemaker Lodge, AK)
			XO.EP23 (Yantarni, AK)
			XO.ET17 (Naknek Lake, AK)
			XO.ET18 (Brooks Camp, AK)
			XO.ET19 (Valley10K Smokes, AK)
			XO.ET20 (Barrier Range, AK)
			XO.KD00 (Pasagshak Bay)
			XO.KD01 (Kalsin Bay)
			XO.KD02 (Anton Larsen Bay, AK)
			XO.KD04 (Santa Flavia Bay, AK)
			XO.KD05 (Uganik, AK)
			XO.KD12 (Anvil Lake, AK)
			XO.KS03 (Chiniak School, AK)
			XO.KS11 (McDonald Lagoon, AK)
			XO.KS13 (Akhiok School, AK)
			XO.KT06 (Harverster Island, AK)
			XO.KT07 (Larsen Bay, AK)
			XO.KT08 (Amook, AK)
			XO.KT09 (Uyak Bay, AK)
			

In [57]:
dfS = obsplus.stations_to_df(inventory)
dfS

,network,station,location,channel,seed_id,latitude,longitude,elevation,depth,azimuth,dip,sample_rate,start_date,end_date
0,XO,EP14,,HHE,XO.EP14..HHE,59.32570,-155.903600,56.0,0.0,90.0,0.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59
1,XO,EP14,,HHN,XO.EP14..HHN,59.32570,-155.903600,56.0,0.0,0.0,0.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59
2,XO,EP14,,HHZ,XO.EP14..HHZ,59.32570,-155.903600,56.0,0.0,0.0,-90.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59
3,XO,EP15,,HHE,XO.EP15..HHE,58.96640,-155.086700,249.0,0.0,90.0,0.0,100.0,2018-06-10 12:00:00,2019-12-31 23:59:59
4,XO,EP15,,HHN,XO.EP15..HHN,58.96640,-155.086700,249.0,0.0,0.0,0.0,100.0,2018-06-10 12:00:00,2019-12-31 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,XO,WS75,,HH2,XO.WS75..HH2,54.17963,-160.256908,-1108.9,0.0,0.0,0.0,100.0,2018-07-18 00:00:00,2019-09-04 23:59:59
295,XO,WS75,,HHZ,XO.WS75..HHZ,54.17963,-160.256908,-1108.9,0.0,0.0,-90.0,100.0,2018-07-18 00:00:00,2019-09-04 23:59:59
296,XO,WS75,,HN1,XO.WS75..HN1,54.17963,-160.256908,-1108.9,0.0,0.0,0.0,100.0,2018-07-18 00:00:00,2019-09-04 23:59:59
297,XO,WS75,,HN2,XO.WS75..HN2,54.17963,-160.256908,-1108.9,0.0,0.0,0.0,100.0,2018-07-18 00:00:00,2019-09-04 23:59:59


In [64]:
station_locs = defaultdict(dict)
for network in inventory:
    for station in network:
        for chn in station:
            sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}"
            print(sid)
            if sid in station_locs:
                station_locs[sid]["component"] += f",{chn.code[-1]}"
                station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
            else:
                component = f"{chn.code[-1]}"
                response = f"{chn.response.instrument_sensitivity.value:.2f}"
                dtype = chn.response.instrument_sensitivity.input_units.lower()
                tmp_dict = {}
                tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation(m)"] = (
                    chn.longitude,
                    chn.latitude,
                    chn.elevation,
                )
                tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                station_locs[sid] = tmp_dict

station_locs = pd.DataFrame.from_dict(station_locs, orient='index')
station_locs["id"] = station_locs.index

NameError: name 'defaultdict' is not defined

In [90]:
s = inventory[0][0]
s

Station EP14 (Igiugig, AK)
	Station Code: EP14
	Channel Count: 3/20 (Selected/Total)
	2018-06-06T00:00:00.000000Z - 2019-12-31T23:59:59.099900Z
	Access: open 
	Latitude: 59.33, Longitude: -155.90, Elevation: 56.0 m
	Available Channels:
		EP14..HHZ, EP14..HHN, EP14..HHE

In [85]:
c = inventory[0][0][0] #[network][station][channel]
c

Channel 'HHE', Location '' 
	Time range: 2018-06-06T00:00:00.000000Z - 2019-12-31T23:59:59.000000Z
	Latitude: 59.33, Longitude: -155.90, Elevation: 56.0 m, Local Depth: 0.0 m
	Azimuth: 90.00 degrees from north, clockwise
	Dip: 0.00 degrees down from horizontal
	Channel types: GEOPHYSICAL
	Sampling Rate: 100.00 Hz
	Sensor (Description): None (Nanometrics Trillium 120 Sec PH Response/Quanterra)
	Response information available

In [86]:
r = c.response
r

Channel Response
	From m/s (velocity in meters per second) to count (digital counts)
	Overall Sensitivity: 5.03939e+08 defined at 0.400 Hz
	3 stages:
		Stage 1: PolesZerosResponseStage from m/s to V, gain: 1201.49
		Stage 2: CoefficientsTypeResponseStage from V to count, gain: 419430
		Stage 3: CoefficientsTypeResponseStage from count to count, gain: 1

In [84]:
r.instrument_sensitivity.input_units.lower()

'm/s'

In [88]:
r.instrument_sensitivity.value

503939000.0

In [94]:
starttime

Timestamp('2018-10-01 03:06:24.332000')

In [97]:
inventory.get_channel_metadata('XO.EP14..HHE')

{'latitude': 59.3257,
 'longitude': -155.9036,
 'elevation': 56.0,
 'local_depth': 0.0,
 'azimuth': 90.0,
 'dip': 0.0}

In [118]:
def get_response_value(inventory, seed_id, start_date):
    ''' given obsplus station dataframe and obspy inventory, get instrument response string '''
    response = inventory.get_response(seed_id, start_date)
    return response.instrument_sensitivity.value

In [119]:
dfS['response'] = dfS.apply(lambda row: get_response_value(inventory, row.seed_id, row.start_date), axis=1)

In [120]:
def get_response_units(inventory, seed_id, start_date):
    ''' given obsplus station dataframe and obspy inventory, get instrument response string '''
    response = inventory.get_response(seed_id, start_date)
    return response.instrument_sensitivity.input_units.lower()

In [121]:
dfS['units'] = dfS.apply(lambda row: get_response_units(inventory, row.seed_id, row.start_date), axis=1)

In [122]:
dfS.head()

,network,station,location,channel,seed_id,latitude,longitude,elevation,depth,azimuth,dip,sample_rate,start_date,end_date,response,units
0,XO,EP14,,HHE,XO.EP14..HHE,59.3257,-155.9036,56.0,0.0,90.0,0.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59,503939000.0,m/s
1,XO,EP14,,HHN,XO.EP14..HHN,59.3257,-155.9036,56.0,0.0,0.0,0.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59,503939000.0,m/s
2,XO,EP14,,HHZ,XO.EP14..HHZ,59.3257,-155.9036,56.0,0.0,0.0,-90.0,100.0,2018-06-06 00:00:00,2019-12-31 23:59:59,503939000.0,m/s
3,XO,EP15,,HHE,XO.EP15..HHE,58.9664,-155.0867,249.0,0.0,90.0,0.0,100.0,2018-06-10 12:00:00,2019-12-31 23:59:59,503939000.0,m/s
4,XO,EP15,,HHN,XO.EP15..HHN,58.9664,-155.0867,249.0,0.0,0.0,0.0,100.0,2018-06-10 12:00:00,2019-12-31 23:59:59,503939000.0,m/s


In [58]:
columns = ['longitude','latitude','elevation','network','sample_rate']
dfS.groupby('station')[columns].first()

,longitude,latitude,elevation,network,sample_rate
station,,,,,
EP14,-155.903600,59.325700,56.0,XO,100.0
EP15,-155.086700,58.966400,249.0,XO,100.0
EP16,-156.856700,59.112600,43.0,XO,100.0
EP21,-157.376900,58.212000,46.0,XO,100.0
EP22,-155.786500,57.900700,63.0,XO,100.0
...,...,...,...,...,...
WS28,-162.299200,55.074600,45.0,XO,100.0
WS71,-153.078922,56.156950,-1554.8,XO,100.0
WS72,-151.810630,56.257748,-4141.7,XO,100.0


In [130]:
list_agg = dfS.groupby(by='station').agg({'channel': lambda x: list(x), 
                                          'response': lambda x: list(x)
                                         },
                                        )
list_agg

,channel,response
station,,
EP14,"[HHE, HHN, HHZ]","[503939000.0, 503939000.0, 503939000.0]"
EP15,"[HHE, HHN, HHZ]","[503939000.0, 503939000.0, 503939000.0]"
EP16,"[HHE, HHN, HHZ]","[503939000.0, 503939000.0, 503939000.0]"
EP21,"[HHE, HHN, HHZ]","[503939000.0, 503939000.0, 503939000.0]"
EP22,"[HHE, HHN, HHZ]","[503939000.0, 503939000.0, 503939000.0]"
...,...,...
WS28,"[HHE, HHN, HHZ, HNE, HNN, HNZ]","[503939000.0, 503939000.0, 503939000.0, 427536..."
WS71,"[HH1, HH2, HHZ, HN1, HN2, HNZ]","[629138000.0, 629138000.0, 629138000.0, 427993..."
WS72,"[HH1, HH2, HHZ, HN1, HN2, HNZ]","[629138000.0, 629138000.0, 629138000.0, 427993..."


## Remote Read

In [23]:
%%time

dfP = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_picks.parquet')
dfE = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_events.parquet')
dfA = pd.read_parquet('https://github.com/scottyhq/quakeflow-examples/raw/main/XO_2018_10_arrivals.parquet')

CPU times: user 215 ms, sys: 28.4 ms, total: 244 ms
Wall time: 1.91 s
